In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
# from matplotlib import pyplot as plt
# # plt.rcParams["figure.figsize"] = (12, 5)
import numpy as np
# from xbout import open_boutdataset
from analysis import *
import os

In [ ]:
# ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data_hires", remove_xgc = True, units="a_mid")
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/DIII-D_runs_5_hires/theta_024/with_churn/sigma_00", remove_xgc = True, units="cm")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/theta_024/umbrella_diff/sigma_05", remove_xgc = True, units="cm")

# Get the rolling average
ds_avg = ds.rolling(t=50, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

In [ ]:
_ = find_nulls(ds, timestamp=-1)

In [ ]:
grid_limited_d_sol(10.0, 0.47/2)

In [13]:
ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
ds_zoomed["B_p"] = np.sqrt(ds_zoomed["B_x"]**2 + ds_zoomed["B_y"]**2) * ds_zoomed.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)
ds["u_mag"] = ds.metadata["C_s0"] * np.sqrt(ds["u_x"]**2 + ds["u_y"]**2)
ds["P"] = ds["P"] * ds.metadata["P_0"]

# Original

In [ ]:
fig,ax = plt.subplots(1)
ds["chi_perp_eff"].transpose().plot()

In [ ]:
##### ANIMATE LIST OF VARIABLES
# ds.bout.animate_list(["P"],ncols=1,cmap="inferno", aspect="equal", logscale=False)
# ds.bout.animate_list(["P_b"],ncols=1,cmap="inferno", aspect="equal", logscale=True)
ds_avg_zoomed.isel(t=range(5,len(ds.t))).bout.animate_list(["u_x", "u_y"],ncols=2,cmap="inferno", aspect="equal", logscale=False)
# ds.boutj.animate_list(["beta_p"],ncols=1,cmap="plasma", aspect="equal", logscale=True, vmin=0.01,vmax=10)
# ds.bout.animate_list(["P_b"],ncols=1,cmap="plasma", aspect="equal", logscale=True)

In [ ]:
# ds_avg.bout.animate_list([ds_avg.metadata["C_s0"]*np.sqrt(ds_avg["u_x"]**2 + ds_avg["u_y"]**2)],ncols=1,cmap="inferno", logscale=False, vmax=3000)
# ds.bout.animate_list([ds["P"].isel(x=int(len(ds.x)/2))],ncols=1)
# ds.bout.animate_list([ds["P"].isel(y=-1)],ncols=1)
ds.bout.animate_list([ds["P"].isel(y=28)],ncols=1, logscale=False)
# ds_zoomed.bout.animate_list([ds_zoomed["B_p"].isel(x=int(len(ds_zoomed.x)/2))],ncols=1)

In [ ]:
# Plot DDY(P)
fig,ax = plt.subplots(1)
levels = [4,5,6,7]
dPdy = ds["P"].differentiate("y").isel(t=-1).values.T
c = ax.contour(ds.x.values, ds.y.values,dPdy,levels=levels)
fig.colorbar(c)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(r"$\partial T / \partial y$")

In [121]:
# Statistical equilibrium
# err = np.sqrt(((ds_avg["P"] - ds_avg["P"].shift(t=1))**2).integrate("x").integrate("y"))
err = abs(ds_avg["P"].differentiate("t").integrate("x").integrate("y"))
fig,ax = plt.subplots(1)
ax.plot(ds.t - ds.t.isel(t=0),err.values)
ax.set_xlabel(r"t [$t_0$]")
ax.set_ylabel(r"$\frac{\partial P}{\partial t}$")
ax.set_yscale("log")
ax.grid()

In [ ]:
animate_q_targets(ds_avg, plot_every=1, normalise=True)
# animate_q_par_targets(ds, plot_every=5)

In [ ]:
q_target_proportions(ds_avg, normalise=True)

In [ ]:
_,q1,q2,q3,q4 = get_Q_legs(ds_avg)
q_tot = q1+q2+q3+q4
fig,ax = plt.subplots(1)
t = (ds.t-ds.t.isel(t=0))*ds.metadata["t_0"] * 1e3
ax.plot(t,q1/q_tot,label="L1")
ax.plot(t,q2/q_tot,label="L2")
ax.plot(t,q3/q_tot,label="L3")
ax.plot(t,q4/q_tot,label="L4")
ax.grid()
ax.set_xlabel("t [ms]")
ax.set_yscale("log")
ax.legend()

In [ ]:
predicted_r_cz(ds)

In [ ]:
# animate_vector(ds_zoomed.isel(t=range(1,len(ds.t))), "u", density=2.0, plot_every=1,arrowsize=0.5, lw_prefactor=5e1)
animate_vector(ds_avg_zoomed.isel(t=range(0,len(ds.t))), "q_cond", fps=5, density=1.0, plot_every=1,arrowsize=0.5, lw_prefactor=1e-6, broken_streamlines=False, savepath="q_cond_avg.gif")
# ntrim=30
# animate_vector(ds.isel(x=range(ntrim,len(ds.x)-ntrim), y=range(ntrim,len(ds.y)-ntrim)), "q_par", plot_every=10, savepath="q_par.gif")

In [108]:
# plot_vector(ds_avg, "q_tot", t=-1, density=2.0, lw_prefactor=3e-8)
plot_vector(ds_avg, "u", t=5, density=2.0, broken_streamlines=True, lw_prefactor=3e1)

In [ ]:
# animate_contour_list(ds,plot_every=2,fps=24, plot_r_cz=False, savepath="osc.gif")
# animate_contour_list(ds, vars=["psi"], plot_every=1,min_timestep=800,max_timestep=None,fps=24,plot_r_cz=False, trim_cells = 35, savepath="churn.gif")
animate_contour_list(ds, vars=["P","psi"], plot_every=1,min_timestep=None,max_timestep=None,fps=10,plot_r_cz=False)

In [ ]:
fig,ax = plt.subplots(1)
ax.grid()
ax.axvline(np.median(ds.x))
ax.axhline(np.median(ds.y))
ax.set_xlim((np.min(ds.x),np.max(ds.x)))
ax.set_ylim((np.min(ds.y),np.max(ds.y)))
ax.contour(ds.x,ds.y,ds.isel(t=0).psi.values.T)

In [ ]:
levels = np.linspace(-0.1,1.1,10)
contour_overlay(ds_zoomed, "P", [-1], fill=True, colorbar=True, levels=levels)
# contour_overlay(ds_avg, "psi", [-1,0], fill=False, colorbar=True, levels=100)
# levels = np.linspace(0,12015.0,10)
# contour_overlay(ds, "P", [-1], fill=True, colorbar=True, levels=levels)
# contour_overlay(ds_zoomed, "psi", [-9,-8,-7,-6,-5,-4,-3,-2,-1], fill=False, colorbar=True, levels=25)
# levels = np.linspace(-0.005,0.000,50)
# contour_overlay(ds_avg_zoomed, "psi", [-1], fill=False, colorbar=True, levels=2000)
# contour_overlay(ds_avg, "P", [-1], fill=True, colorbar=True, levels=[0,1,2,3])
# contour_overlay(ds, "beta_p", [-1], fill=True, levels=[0,1,3,10], colorbar=True)

In [ ]:
plot_conservation(ds)

In [ ]:
predicted_r_cz(ds) * ds.metadata["a_mid"] * 100

# 27/11/2024 analysis

In [ ]:
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_0p5/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_1p0/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_5p0/eps_0p00"]
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_0p5/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_1p0/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p00"]
rundirs = [r"/Users/power8/Documents/07_churning_mode/beta_p_scan_longrun/P_core_0p5/eps_0p00",
           r"/Users/power8/Documents/07_churning_mode/beta_p_scan_longrun/P_core_1p0/eps_0p00",
           r"/Users/power8/Documents/07_churning_mode/beta_p_scan_longrun/P_core_5p0/eps_0p00"]

P_cores = [0.5,1.0,5.0]
beta_ps = []
Q1s = []
Q2s = []
Q3s = []
Q4s = []

for rundir in rundirs:
    ds = read_boutdata(os.path.join(rundir, "data"), remove_xgc=True)
    # Get the rolling average
    ds_avg = ds.rolling(t=5, min_periods=1).mean()
    ds_avg_zoomed = ds_avg.isel(
        x=range(int(len(ds.x) / 4), 3 * int(len(ds.x) / 4)),
        y=range(int(len(ds.y) / 4), 3 * int(len(ds.y) / 4)),
    )

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds)
    Q_tot = Q1 + Q2 + Q3 + Q4
    Q1s.append(100*(Q1/Q_tot).isel(t=-1).values)
    Q2s.append(100*(Q2/Q_tot).isel(t=-1).values)
    Q3s.append(100*(Q3/Q_tot).isel(t=-1).values)
    Q4s.append(100*(Q4/Q_tot).isel(t=-1).values)

    beta_ps.append(ds.metadata["beta_p"])
beta_ps = [P_cores[i] * beta_ps[i] for i in range(len(P_cores))]

fig,ax = plt.subplots(1)
ax.stackplot(beta_ps, [Q1s,Q2s,Q3s,Q4s], labels=["Leg 1", "Leg 2", "Leg 3", "Leg 4"])
ax.legend()
ax.set_xlabel(r"$\beta_{pmid}$")
ax.set_ylabel(r"Power fraction [%]")

In [ ]:
ds1 = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_1p0/eps_0p00/data", remove_xgc = True)
ds2 = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_1p0/eps_0p00/data", remove_xgc = True)

Py1 = ds1["P"].isel(x=int(len(ds1.x)/2),t=0)
Py2 = ds2["P"].isel(x=int(len(ds2.x)/2),t=0)

r_cz1 = predicted_r_cz(ds1) * ds1.metadata["a_mid"]
r_cz2 = predicted_r_cz(ds2) * ds2.metadata["a_mid"]

midpoint = ds1.y.isel(y=int(len(ds1.y)/2)).values * ds1.metadata["a_mid"]

fig,ax = plt.subplots(1)
ax.plot(ds1.y * ds1.metadata["a_mid"],Py1, label=r"$\chi_{\perp}=1.0m^2s^-1$")
ax.plot(ds2.y * ds2.metadata["a_mid"],Py2, label=r"$\chi_{\perp}=0.1m^2s^-1$")
ax.legend()
ax.grid()
ax.set_ylabel("$P$ [$P_0$]")
ax.set_xlabel("$y$ [cm]")
ax.axvline(1.15*midpoint+r_cz1, linestyle="--", color="gray")
ax.axvline(1.15*midpoint-r_cz1, linestyle="--", color="gray")
fig.tight_layout()

In [ ]:
rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p00",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p02",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p08"]

r_seps = [0.0,7.0,14.0]
Q1s = []
Q2s = []
Q3s = []
Q4s = []
Q1s_0 = []
Q2s_0 = []
Q3s_0 = []
Q4s_0 = []

for rundir in rundirs:
    ds = read_boutdata(os.path.join(rundir, "data"), remove_xgc=True)
    # Get the rolling average
    ds_avg = ds.rolling(t=10, min_periods=1).mean()
    ds_avg_zoomed = ds_avg.isel(
        x=range(int(len(ds.x) / 4), 3 * int(len(ds.x) / 4)),
        y=range(int(len(ds.y) / 4), 3 * int(len(ds.y) / 4)),
    )

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds)
    Q_tot = Q1 + Q2 + Q3 + Q4
    Q1s.append(100*(Q1/Q_tot).isel(t=-1).values)
    Q2s.append(100*(Q2/Q_tot).isel(t=-1).values)
    Q3s.append(100*(Q3/Q_tot).isel(t=-1).values)
    Q4s.append(100*(Q4/Q_tot).isel(t=-1).values)
    Q1s_0.append(100*(Q1/Q_tot).isel(t=0).values)
    Q2s_0.append(100*(Q2/Q_tot).isel(t=0).values)
    Q3s_0.append(100*(Q3/Q_tot).isel(t=0).values)
    Q4s_0.append(100*(Q4/Q_tot).isel(t=0).values)

Q1s = np.array(Q1s)
Q2s = np.array(Q2s)
Q3s = np.array(Q3s)
Q4s = np.array(Q4s)
Q1s_0 = np.array(Q1s_0)
Q2s_0 = np.array(Q2s_0)
Q3s_0 = np.array(Q3s_0)
Q4s_0 = np.array(Q4s_0)

fig,ax = plt.subplots(1)
ax.plot(r_seps, Q3s/100, color='red', label="with churn")
ax.plot(r_seps, Q3s_0/100, color="black", label="without churn")
# ax2 = ax.twinx()
# ax2.plot(r_seps, Q2s / Q2s_0, color="gray", linestyle="--", label="ratio")
ax.legend()
# ax2.legend(loc="lower right")
# ax2.set_ylabel("ratio")
ax.set_xlabel(r"$r_{sep}$ [cm]")
ax.set_ylabel(r"Power fraction")
ax.grid()
ax.set_yscale("log")
ax.set_title("Leg 3")
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(3)
ax[0].plot(r_seps, Q2s/100, color='red', label="Leg 2 (=SP3)")
ax[1].plot(r_seps, Q3s/100, color='red', label="Leg 3 (=SP2)")
ax[2].plot(r_seps, Q4s/100, color='red', label="Leg 4 (=SP1)")
ax[0].plot(r_seps, Q2s_0/100, color='black', linestyle="--")
ax[1].plot(r_seps, Q3s_0/100, color='black', linestyle="--")
ax[2].plot(r_seps, Q4s_0/100, color='black', linestyle="--")
ax[0].grid()
ax[1].grid()
ax[2].grid()
# ax.plot(r_seps, Q3s_0/100, color="black", label="without churn")
ax[0].legend(loc="upper left")
ax[1].legend(loc="upper left")
ax[2].legend(loc="upper left")
# ax2.legend(loc="lower right")
# ax2.set_ylabel("ratio")
ax[-1].set_xlabel(r"$r_{sep}$ [cm]")
# ax.set_ylabel(r"Power fraction")
ax[0].set_yscale("log")
ax[1].set_yscale("log")
ax[0].set_ylim((0.01,0.3))
ax[1].set_ylim((0.01,0.2))
ax[2].set_ylim((0,0.7))
ax2 = ax[2].twinx()
ax2.plot([],[],color="red", label="With churn")
ax2.plot([],[],color="black", linestyle="--", label="Without churn")
ax2.legend(loc="lower left")
ax2.set_yticks([])
# ax[0].set_title("Leg 3")
fig.tight_layout()

# 4/12/2024 analysis

In [ ]:
# Topology change
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/topo_reconfig/TCV/P_0_30x", remove_xgc = True)

# Get the rolling average
ds_avg = ds.rolling(t=20, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

levels = np.array([-0.5,0.00005,0.00479,0.5])
contour_overlay(ds_avg, "psi", [0], fill=True, levels=levels)

# levels = np.array([-0.5,0.00166,0.00387,0.5])
levels = np.array([-0.5,0.00387,0.00891,0.5])
contour_overlay(ds_avg, "psi", [-1], fill=True, levels=levels)

In [ ]:
rundirs = [r"/Users/power8/Documents/07_churning_mode/topo_reconfig_investigation/TCV/P_0_10x",
           r"/Users/power8/Documents/07_churning_mode/topo_reconfig_investigation/TCV/P_0_30x",
           r"/Users/power8/Documents/07_churning_mode/topo_reconfig_investigation/TCV/P_0_50x"]

multipliers = np.array([10,30,50])
ratios_t0 = []
ratios_t1 = []
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds_avg = ds.rolling(t=20, min_periods=1).mean()

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    ratios_t0.append((Q2 / Q4).isel(t=0).values)
    ratios_t1.append((Q2 / Q4).isel(t=-1).values)
    q_target_proportions(ds_avg, normalise=True)

ratios_t0 = np.array(ratios_t0)
ratios_t1 = np.array(ratios_t1)

fig,ax = plt.subplots(1)
# ax.plot(multipliers,ratios_t0,linestyle="--",marker="x")
ax.axhline(ratios_t0[0], linestyle="--", color="gray",label="inter ELM")
ax.plot(np.sqrt(multipliers), ratios_t1, linestyle="--", marker="x", label="ELM")
ax.set_ylabel("$P_3 / P_1$")
ax.set_xlabel(r"ELM $\beta_p$ multiplier")
ax.legend()
ax.grid()

# 11/12/2024 analysis

In [ ]:
# Match DIII-D experimental beta_p profiles
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_2/with_churn/sigma_01", remove_xgc = True)

# Get the rolling average
ds_avg = ds.rolling(t=10, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
ds_zoomed["B_p"] = np.sqrt(ds_zoomed["B_x"]**2 + ds_zoomed["B_y"]**2) * ds_zoomed.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)

lambda_sol = 0.003
D_expansion = ds.metadata["a_mid"] * (lambda_sol/ds.metadata["a_mid"])**(1/3)
f_exp = D_expansion/lambda_sol
print(f_exp)

timestamp = 0

colours = ["red", "blue", "green"]

fig,ax = plt.subplots(3)
offsets = ([3,2],[3,4],[3,6])
for i in range(len(offsets)):
    xcoord = int(len(ds.x)/2)+offsets[i][0]
    ycoord = int(len(ds.x)/2)+offsets[i][1]
    beta_p = ds["beta_p"].isel(t=timestamp, y=ycoord)
    P_e = 0.5 * 10 * ds.metadata["P_0"] * ds["P"].isel(t=timestamp, y=ycoord)
    P_b = ds["P_b"].isel(t=timestamp, y=ycoord)
    rho = ds["rho"].isel(t=timestamp, y=ycoord)
    xaxis = (ds.x - ds.x[xcoord]) * 10/f_exp
    # xaxis = -rho
    ax[0].scatter(xaxis,P_e, color=colours[i])
    ax[1].scatter(xaxis,P_b, color=colours[i])
    ax[2].scatter(xaxis,beta_p, color=colours[i])

    print("{}, x:{:.1f}, y:{:.1f}".format(colours[i],ds.x[int(len(ds.x)/2+1)],ds.y[ycoord]))

ax[0].set_yscale("log")
ax[0].set_ylim([500,3e5])
ax[0].set_xlim([-1,6])
ax[0].grid()
ax[0].set_ylabel("$P_e$ [dyn cm$^{-2}$]")

ax[1].set_yscale("log")
ax[1].set_ylim([100,10000])
ax[1].set_xlim([-1,6])
ax[1].grid()
ax[1].set_ylabel("$P_b$ [dyn cm$^{-2}$]")

ax[2].set_yscale("log")
ax[2].set_ylim([0.1,400])
ax[2].set_xlim([-1,6])
ax[2].grid()
ax[2].set_ylabel(r"$\beta_p$")

In [ ]:
# Match DIII-D EFIT beta_p profiles
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_2/with_churn/sigma_01", remove_xgc = True)
# ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True)
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_01", remove_xgc = True)

# Get the rolling average
ds_avg = ds.rolling(t=10, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
ds_zoomed["B_p"] = np.sqrt(ds_zoomed["B_x"]**2 + ds_zoomed["B_y"]**2) * ds_zoomed.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 100 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)

# timestamp = 0
timestamp = 0

colours = ["red", "blue", "green"]

fig,ax = plt.subplots(3)
offsets = [[4,2]]
for i in range(len(offsets)):
    xcoord = int(len(ds.x)/2)+offsets[i][0]
    ycoord = int(len(ds.y)/2)+offsets[i][1]
    beta_p = ds["beta_p"].isel(t=timestamp, x=xcoord)
    P_e = 0.5 * 10 * ds.metadata["P_0"] * ds["P"].isel(t=timestamp, x=xcoord)
    P_b = ds["P_b"].isel(t=timestamp, x=xcoord)
    rho = ds["rho"].isel(t=timestamp, x=xcoord)
    xaxis = ds.y
    # xaxis = -rho
    ax[0].scatter(xaxis,P_e, color=colours[i])
    ax[1].scatter(xaxis,P_b, color=colours[i])
    ax[2].scatter(xaxis,beta_p, color=colours[i])

    print("{}, x:{:.1f}, y:{:.1f}".format(colours[i],ds.x[xcoord],ds.y[ycoord]))

ax[0].set_yscale("log")
ax[0].set_ylim([0.7e4,7e5])
ax[0].set_xlim([ds.y[int(len(ds.y)/2)]-4.0,ds.y[int(len(ds.y)/2)]+25])
ax[0].grid()
ax[0].set_ylabel("$P_e$ [dyn cm$^{-2}$]")

ax[1].set_yscale("log")
ax[1].set_ylim([50,9e3])
ax[1].set_xlim([ds.y[int(len(ds.y)/2)]-4.0,ds.y[int(len(ds.y)/2)]+25])
ax[1].grid()
ax[1].set_ylabel("$P_b$ [dyn cm$^{-2}$]")

ax[2].set_yscale("log")
ax[2].set_ylim([10,400])
ax[2].set_xlim([ds.y[int(len(ds.y)/2)]-4.0,ds.y[int(len(ds.y)/2)]+25])
ax[2].grid()
ax[2].set_ylabel(r"$\beta_p$")
ax[2].set_xlabel("y [cm]")

In [ ]:
ds.bout.animate_list(["P_b"],logscale=True)

In [ ]:
# Angle change
rundirs = [r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p0",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p2",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p4",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p6",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p8",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p0",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p2",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p4",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p6"]

# x1_is = [[57,66], [57.2,68.3]]
# x2_is = [[83.6,66], [83.5,63.7]]
# x1_fs = [[81.7,66.7], [86.8,59.2]]
# x2_fs = [[54.9,66.6], [54.3,69.9]]

Q1_is =[]
Q2_is =[]
Q3_is =[]
Q4_is =[]
Q1_fs =[]
Q2_fs =[]
Q3_fs =[]
Q4_fs =[]
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds_avg = ds.rolling(t=10, min_periods=1).mean()

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    Qtot = Q1 + Q2 + Q3 + Q4
    Q1_is.append((Q1/Qtot).isel(t=0))
    Q2_is.append((Q2/Qtot).isel(t=0))
    Q3_is.append((Q3/Qtot).isel(t=0))
    Q4_is.append((Q4/Qtot).isel(t=0))
    Q1_fs.append((Q1/Qtot).isel(t=-1))
    Q2_fs.append((Q2/Qtot).isel(t=-1))
    Q3_fs.append((Q3/Qtot).isel(t=-1))
    Q4_fs.append((Q4/Qtot).isel(t=-1))


In [ ]:
angles = [0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6]

fig,ax = plt.subplots(4, sharex=True)
ax[0].plot(angles, Q1_is, color="black", linestyle="--")
ax[0].plot(angles, Q1_fs, color="red", linestyle="--")
ax[0].set_ylim([1e-1,1.0])
ax[0].set_yscale("log")
ax[0].set_ylabel("$P_1$")

ax[1].plot(angles, Q2_is, color="black", linestyle="--")
ax[1].plot(angles, Q2_fs, color="red", linestyle="--")
ax[1].set_ylim([1e-4,0.1])
ax[1].set_yscale("log")
ax[1].set_ylabel("$P_2$")

ax[2].plot(angles, Q3_is, color="black", linestyle="--")
ax[2].plot(angles, Q3_fs, color="red", linestyle="--")
ax[2].set_ylim([1e-4,0.1])
ax[2].set_yscale("log")
ax[2].set_ylabel("$P_3$")

ax[3].plot(angles, Q4_is, color="black", linestyle="--")
ax[3].plot(angles, Q4_fs, color="red", linestyle="--")
ax[3].set_ylim([1e-1,1.0])
ax[3].set_yscale("log")
ax[3].set_xlabel(r"$\theta$")
ax[3].set_ylabel("$P_4$")


In [ ]:
# Replicate fig 6 in DIII-D paper (SP3,2,1 power fraction vs r_sep)
rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_00",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_01",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_02",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_03"]
        #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_04",
        #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_05"]


# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])
# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0])
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5])
P1s = []
P2s = []
P3s = []
timestamp = -1
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds_avg = ds.rolling(t=100, min_periods=1).mean()

    _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    Q_tot = Q1 + Q2 + Q3 + Q4
    P1s.append((Q1/Q_tot).isel(t=timestamp))
    P2s.append((Q2/Q_tot).isel(t=timestamp))
    P3s.append((Q3/Q_tot).isel(t=timestamp))

fig,ax = plt.subplots(3,sharex=True)
ax[0].plot(r_sep,P3s, color="red", marker="x", linestyle="--")
ax[0].set_yscale("log")
ax[0].set_ylim([0.0095,0.11])
ax[0].set_ylabel("$P_3$")
ax[0].grid()
ax[1].plot(r_sep,P2s, color="red", marker="x", linestyle="--")
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.25])
ax[1].set_ylabel("$P_2$")
ax[1].grid()
ax[2].plot(r_sep,P1s, color="red", marker="x", linestyle="--")
ax[2].set_ylim([0.0,0.71])
ax[2].set_ylabel("$P_1$")
ax[2].set_xlim([-1,30])
ax[2].grid()
ax[2].set_xlabel("$D_{xx}$ [cm]")

In [ ]:
# Time series of power to each leg vs. D_xx
rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_00",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_01",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_02",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_03",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_04",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_05"]
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_02",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_03",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_03",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_04",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_05"]
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_01",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_02",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_03"]
#         #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_04",
#         #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_05"]

# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5])
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5, 2.0,2.5])
# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])

# timestamps = [0,50,100,150,200]
timestamps = [-100, -50,-1]
tgrid = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_00", remove_xgc = True).t

P1s = []
P2s = []
P3s = []
P4s = []
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds = ds.interp(t=tgrid)
    ds_avg = ds.rolling(t=50, min_periods=1).mean()

    _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    Q_tot = Q1 + Q2 + Q3 + Q4
    P1s.append([])
    P2s.append([])
    P3s.append([])
    P4s.append([])
    for timestamp in timestamps:
        P1s[-1].append((Q1/Q_tot).isel(t=timestamp))
        P2s[-1].append((Q2/Q_tot).isel(t=timestamp))
        P3s[-1].append((Q3/Q_tot).isel(t=timestamp))
        P4s[-1].append((Q4/Q_tot).isel(t=timestamp))

P1s = np.array(P1s).T
P2s = np.array(P2s).T
P3s = np.array(P3s).T
P4s = np.array(P4s).T

alphas = np.linspace(0.25,1.0,len(timestamps))

fig,ax = plt.subplots(4,sharex=True)
for i in range(len(timestamps)):
    ax[0].plot(r_sep,P4s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[0].set_ylim([0.0,0.71])
ax[0].set_ylabel("$P_4$")
ax[0].set_xlim([-1,30])
ax[0].grid()
for i in range(len(timestamps)):
    ax[1].plot(r_sep,P3s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.11])
ax[1].set_ylabel("$P_3$")
ax[1].grid()
for i in range(len(timestamps)):
    ax[2].plot(r_sep,P2s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[2].set_yscale("log")
ax[2].set_ylim([0.0095,0.25])
ax[2].set_ylabel("$P_2$")
ax[2].grid()
for i in range(len(timestamps)):
    ax[3].plot(r_sep,P1s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[3].set_ylim([0.0,0.71])
ax[3].set_ylabel("$P_1$")
ax[3].set_xlim([-1,30])
ax[3].grid()
ax[3].set_xlabel("$D_{xx}$ [cm]")

for i in range(len(timestamps)):
    ax[0].plot([],[],color="red",linestyle="--",marker="x",alpha=alphas[i],label="t={:.2f}ms".format(1000* (tgrid[timestamps[i]]-tgrid[0]) * ds.metadata["t_0"]))
ax[0].legend(loc="upper right")

In [ ]:
# Plot psi using parameters from EFIT fit
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/EFIT_comparison/DIII-D_fitted_psi/data", remove_xgc = True)
fig,ax = plt.subplots(1)
levels = np.linspace(0.031415719597447156, 0.052115647479612406,50)
# levels=50
c = ax.contour(ds.x, ds.y, ((ds["psi"]*ds.metadata["psi_0"]) + (1.07e-03*3.91e+01)).isel(t=-1).values.T, levels=levels, cmap="inferno")
# c = ax.pcolormesh(ds.x, ds.y, ((ds["psi"]*ds.metadata["psi_0"]) + (1.07e-03*3.91e+01)).isel(t=-1).values.T, cmap="inferno")
fig.colorbar(c)
ax.set_xlabel("R [m]")
ax.set_ylabel("Z [m]")

In [ ]:
# Plot pressure along same line as DTS EFIT measurement in DIII-D paper (fig. 11)
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/EFIT_comparison/DIII-D_fitted_P/data", remove_xgc = True)
ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_02", remove_xgc = True)
# ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
# mu_0 = 1.256637e-6
# ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)
ds["P_el"] = 0.5 * 10 * ds["P"] * ds.metadata["P_0"]
ds["x"] = (ds.x-np.median(ds.x))
ds["y"] = (ds.y-np.median(ds.y))
# ds = ds.rolling(t=50,min_periods=1).mean()

fig,ax = plt.subplots(ncols=2,nrows=1)

alpha = 0.25
x_rot = ds.x * np.cos(alpha) - ds.y*np.sin(alpha)
y_rot = ds.x * np.sin(alpha) + ds.y*np.cos(alpha)

ds_rot = ds.interp(x=x_rot,y=y_rot)
timestamp = -1
# timestamp = 0
# ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=[-0.000353,0], colors="white")
# ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=np.linspace(-0.04199,-0.041635,2), colors="white")
ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=np.linspace(-0.00035,0.00,2), colors="white")
ax[0].pcolormesh(ds_rot.x,ds_rot.y,ds_rot["P"].isel(t=timestamp).values.T, cmap="inferno")
# x_idx = find_nearest(ds_rot.x, 7.0)
# y_idx = find_nearest(ds_rot.y, -2.5)
x_1 = 0.093 # x-coordinate of first X-point
y_1 = -0.038 # y-coordinate of first X-point
x_idx = find_nearest(ds_rot.x, x_1)
y_idx = find_nearest(ds_rot.y, y_1)
ax[0].axvline(x=ds_rot.x.isel(x=x_idx), color="red")

ax[1].plot(ds_rot.y.isel(y=range(y_idx,len(ds_rot.y))),ds_rot["P_el"].isel(x=x_idx,t=timestamp,y=range(y_idx,len(ds_rot.y))), color="red", marker="x")
ax[1].set_xlabel("y [cm]")
ax[1].set_ylabel("$P_{el}$")
ax[1].set_yscale("log")
ax[1].grid()
# ax[1].set_ylim([0.8e4,0.7e6])
# ax[1].set_xlim(min(ds_rot.y.isel(y=range(y_idx,len(ds_rot.y)))),min(ds_rot.y.isel(y=range(y_idx,len(ds_rot.y))))+27)
# ax[1].axvline(x=7.0,linestyle="--",color="gray")
fig.tight_layout()

# 18/12/2024 analysis

In [ ]:
# Plot P_l vs. D_xx for different null orientations
sigmas = ["sigma_00", "sigma_01", "sigma_02", "sigma_03"]
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5])

# th1_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_3/with_churn"
# th2_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_3_theta2/with_churn"
# th3_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_3_theta3/with_churn"
# rundirs  = [th1_rundir, th2_rundir, th3_rundir]
th1_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn"
th2_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_075/with_churn"
th3_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_150/with_churn"
th4_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_m025/with_churn"
th5_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_m075/with_churn"
rundirs  = [th1_rundir, th2_rundir, th3_rundir, th4_rundir, th5_rundir]
thetas = [r"$\theta=14^{\degree}$", r"$\theta=43^{\degree}$", r"$\theta=86^{\degree}$", r"$\theta=-14^{\degree}$", r"$\theta=-43^{\degree}$"]
# rundirs  = [th1_rundir]
# thetas = [r"$\theta=14^{\degree}$"]

tgrid = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_075/with_churn/sigma_00", remove_xgc = True).t

markers=["x", "+", "*", "o", "v"]

timestamp = 0
avg_window = 50
P1s = np.zeros((len(rundirs),len(sigmas)))
P2s = np.zeros((len(rundirs),len(sigmas)))
P3s = np.zeros((len(rundirs),len(sigmas)))
P4s = np.zeros((len(rundirs),len(sigmas)))
for i,rundir in enumerate(rundirs):
    for j,sigma in enumerate(sigmas):
        ds = read_boutdata(os.path.join(rundir,sigma), remove_xgc = True)
        ds.interp(t=tgrid)
        ds_avg = ds.rolling(t=avg_window, min_periods=1).mean()

        _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
        Q_tot = Q1 + Q2 + Q3 + Q4
        P1s[i,j] = (Q1/Q_tot).isel(t=timestamp)
        P2s[i,j] = (Q2/Q_tot).isel(t=timestamp)
        P3s[i,j] = (Q3/Q_tot).isel(t=timestamp)
        P4s[i,j] = (Q4/Q_tot).isel(t=timestamp)

alphas = np.linspace(1.0,0.25,len(rundirs))

# fig,ax = plt.subplots(3,sharex=True)
fig,ax = plt.subplots(4,sharex=True)
for i in range(len(rundirs)):
    ax[0].plot(r_sep,P4s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[0].set_ylim([0.0,0.71])
ax[0].set_ylabel("$P_4$")
ax[0].set_xlim([-1,30])
ax[0].grid()
for i in range(len(rundirs)):
    ax[1].plot(r_sep,P3s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.11])
# ax[1].set_ylim([0.0065,0.3])
ax[1].set_ylabel("$P_3$")
ax[1].grid()
for i in range(len(rundirs)):
    ax[2].plot(r_sep,P2s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[2].set_yscale("log")
ax[2].set_ylim([0.0095,0.25])
ax[2].set_ylabel("$P_2$")
ax[2].grid()
for i in range(len(rundirs)):
    ax[3].plot(r_sep,P1s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[3].set_ylim([-0.03,0.73])
ax[3].set_ylabel("$P_1$")
ax[3].set_xlim([-1,30])
ax[3].grid()
ax[3].set_xlabel("Inter-null separation [cm]")

for i in range(len(rundirs)):
    ax[0].plot([],[],color="red", marker=markers[i], linestyle="--", alpha=alphas[i], label=thetas[i])
ax[0].legend(loc="upper right")

In [ ]:
# Plot P_l vs. D_xx for different null orientations
sigmas = ["sigma_00", "sigma_01", "sigma_02", "sigma_03", "sigma_04", "sigma_05"]
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])

res1_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn"
res2_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn"
rundirs  = [res1_rundir, res2_rundir]
resolutions = [r"64x40 cells", r"256x160 cells"]

tgrid = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_00", remove_xgc = True).t

markers=["x", "+", "*", "o", "v"]

# timestamp = -1
# avg_window = 50
# P1s = np.zeros((len(rundirs),len(sigmas)))
# P2s = np.zeros((len(rundirs),len(sigmas)))
# P3s = np.zeros((len(rundirs),len(sigmas)))
# P4s = np.zeros((len(rundirs),len(sigmas)))
# for i,rundir in enumerate(rundirs):
#     for j,sigma in enumerate(sigmas):
#         ds = read_boutdata(os.path.join(rundir,sigma), remove_xgc = True)
#         ds.interp(t=tgrid)
#         ds_avg = ds.rolling(t=avg_window, min_periods=1).mean()

#         _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
#         Q_tot = Q1 + Q2 + Q3 + Q4
#         P1s[i,j] = (Q1/Q_tot).isel(t=timestamp)
#         P2s[i,j] = (Q2/Q_tot).isel(t=timestamp)
#         P3s[i,j] = (Q3/Q_tot).isel(t=timestamp)
#         P4s[i,j] = (Q4/Q_tot).isel(t=timestamp)

alphas = np.linspace(1.0,0.25,len(rundirs))

fig,ax = plt.subplots(4,sharex=True)
for i in range(len(rundirs)):
    ax[0].plot(r_sep,P4s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[0].set_ylim([0.0,0.71])
ax[0].set_ylabel("$P_4$")
ax[0].set_xlim([-1,30])
ax[0].grid()
for i in range(len(rundirs)):
    ax[1].plot(r_sep,P3s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.11])
# ax[1].set_ylim([0.0065,0.3])
ax[1].set_ylabel("$P_3$")
ax[1].grid()
for i in range(len(rundirs)):
    ax[2].plot(r_sep,P2s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[2].set_yscale("log")
ax[2].set_ylim([0.0095,0.25])
ax[2].set_ylabel("$P_2$")
ax[2].grid()
for i in range(len(rundirs)):
    ax[3].plot(r_sep,P1s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[3].set_ylim([-0.03,0.73])
ax[3].set_ylabel("$P_1$")
ax[3].set_xlim([-1,30])
ax[3].grid()
ax[3].set_xlabel("Inter-null separation [cm]")

for i in range(len(rundirs)):
    ax[0].plot([],[],color="red", marker=markers[i], linestyle="--", alpha=alphas[i], label=resolutions[i])
ax[0].legend(loc="upper right")

In [ ]:
# Plot psi along same line as DTS EFIT measurement in DIII-D paper (fig. 11)
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/EFIT_comparison/DIII-D_fitted_P/data", remove_xgc = True)
ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True)
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_02", remove_xgc = True)
ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)
ds["P_el"] = 0.5 * 10 * ds["P"] * ds.metadata["P_0"]
ds["x"] = (ds.x-np.median(ds.x))
ds["y"] = (ds.y-np.median(ds.y))
ds = ds.rolling(t=50,min_periods=1).mean()

fig,ax = plt.subplots(ncols=2,nrows=1)

alpha = 0.25
x_rot = ds.x * np.cos(alpha) - ds.y*np.sin(alpha)
y_rot = ds.x * np.sin(alpha) + ds.y*np.cos(alpha)

ds_rot = ds.interp(x=x_rot,y=y_rot)
timestamp = -1
# timestamp = 0
# ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=[-0.000353,0], colors="white")
ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=np.linspace(-0.04199,-0.041635,2), colors="white")
ax[0].pcolormesh(ds_rot.x,ds_rot.y,ds_rot["P"].isel(t=timestamp).values.T, cmap="inferno")
# x_idx = find_nearest(ds_rot.x, 7.0)
# y_idx = find_nearest(ds_rot.y, -2.5)
x_1 = 0.0070 # x-coordinate of first X-point  [a_mid]
y_1 = -0.0122 # y-coordinate of first X-point  [a_mid]
x_idx = find_nearest(ds_rot.x, x_1)
y_idx = find_nearest(ds_rot.y, y_1)
ax[0].axvline(x=ds_rot.x.isel(x=x_idx), color="red")
ax[0].axhline(y=ds_rot.y.isel(y=y_idx), color="blue")


psi_ds = ds_rot.isel(x=x_idx,t=timestamp,y=range(y_idx,len(ds_rot.y)))["psi"]
psi_ds["y"] = (ds_rot.y.isel(y=range(y_idx,len(ds_rot.y)))-ds_rot.y.isel(y=y_idx)-125)/100
ax[1].plot(psi_ds.y,psi_ds, color="red", marker="x")
ax[1].set_xlabel("y [cm]")
ax[1].set_ylabel("$psi$")
ax[1].grid()
# ax[1].axvline(x=5.5,linestyle="--",color="gray")
fig.tight_layout()


fig11_coords = [-1.2453719008264463, -1.2322314049586778,-1.2076859504132231,-1.1707438016528926,-1.1310743801652894,-1.0921487603305784,-1.0544628099173554,-1.0118181818181817]
psi_fig11 = psi_ds.interp(y=fig11_coords).values
print(psi_fig11)

In [ ]:
# Compare umbrella diffusion method vs turbulent calculation

sigmas = ["sigma_00", "sigma_01", "sigma_02", "sigma_03", "sigma_04", "sigma_05"]
r_seps = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])
# sigmas = ["sigma_00", "sigma_01"]
# r_seps = 11.4 * np.array([0.1,0.5])

turb_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn"
diff_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_umbrella/theta_024/with_churn"
# turb_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_075/with_churn"
# diff_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_umbrella/theta_075/with_churn"
rundirs  = [turb_rundir, diff_rundir]
resolutions = ["turbulent calc", "umbrella diff"]

plot_power_deposition_vs_d_sep(rundirs=rundirs, 
                               rundir_labels = resolutions, 
                               sepdirs = sigmas, 
                               seps=r_seps, 
                               rundir_colours=["red", "black"],
                               plot_p1=False,
                               plot_p4=False)